# Min-dalle classes

In [ ]:
import os
if not os.path.isfile('/content/once.txt'):
  !pip install accelerate
  !apt-get install -y libvulkan-dev libomp5
  !pip install ncnn-vulkan
  !mv /usr/local/lib/python3.7/dist-packages/ncnn_vulkan/*.so /usr/local/lib/python3.7/dist-packages/ncnn/
  !pip install emoji
  !wget -O /tmp/vq.param https://raw.githubusercontent.com/TabuaTambalam/vqqncnn/main/vq.param
  !wget -O /tmp/vq.bin https://github.com/TabuaTambalam/vqqncnn/releases/download/0.0/vq.bin
  !wget -O /tmp/vq_vert.param https://raw.githubusercontent.com/TabuaTambalam/vqqncnn/main/vq_vert.param
  !git clone https://github.com/kuprel/min-dalle.git
  !mv 'min-dalle/min_dalle' min_dalle
  !rm -rf '/content/min-dalle'

In [ ]:
%%writefile /content/once.txt

2@0

Writing /content/once.txt


In [ ]:
from PIL import Image
from threading import Thread
import sys
import numpy as np
from torch import LongTensor, FloatTensor, BoolTensor,nn
from math import sqrt
import torch
import torch.backends.cudnn, torch.backends.cuda
import json
import requests
from typing import Iterator, List, Tuple, Dict
from min_dalle.text_tokenizer import TextTokenizer
from min_dalle.models import DalleBartEncoder, DalleBartDecoder

torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

MIN_DALLE_REPO = 'https://huggingface.co/kuprel/min-dalle/resolve/main/'
IMAGE_TOKEN_COUNT = 256


from accelerate import init_empty_weights

def get_keys_to_submodule(model: nn.Module) -> Dict[str, nn.Module]:
    keys_to_submodule = {}
    # iterate all submodules
    for submodule_name, submodule in model.named_modules():
        # iterate all paramters in each submobule
        for param_name, param in submodule.named_parameters():
            # param_name is organized as <name>.<subname>.<subsubname> ...
            # the more we go deep in the model, the less "subname"s we have
            splitted_param_name = param_name.split('.')
            # if we have only one subname, then it means that we reach a "leaf" submodule, 
            # we cannot go inside it anymore. This is the actual parameter
            is_leaf_param = len(splitted_param_name) == 1
            if is_leaf_param:
                # we recreate the correct key
                key = f"{submodule_name}.{param_name}"
                # we associate this key with this submodule
                keys_to_submodule[key] = submodule
                
    return keys_to_submodule

def load_state_dict_with_low_memory(model: nn.Module, state_dict):
    print('======hacky load======')
    # free up memory by placing the model in the `meta` device
    keys_to_submodule = get_keys_to_submodule(model)
    mste=model.state_dict()
    for key, submodule in keys_to_submodule.items():
        # get the valye from the state_dict
        if key in state_dict:
          val = state_dict[key]
        else:
          val = torch.ones(mste[key].shape, dtype= torch.float16)
        # we need to substitute the parameter inside submodule, 
        # remember key is composed of <name>.<subname>.<subsubname>
        # the actual submodule's parameter is stored inside the 
        # last subname. If key is `in_proj.weight`, the correct field if `weight`
        param_name = key.split('.')[-1]
        #param_dtype = getattr(submodule, param_name).dtype
        #val = val.to(param_dtype)
        # create a new parameter
        new_val = torch.nn.Parameter(val)
        setattr(submodule, param_name, new_val)


def init2(mdl,path):
  load_state_dict_with_low_memory(mdl, torch.load(path,map_location='cpu'))
  mdl.requires_grad_(False).eval()
  


class MinDalle:
    def __init__(
        self,
        models_root: str = 'pretrained',
        dtype: torch.dtype = torch.float16,
        device: str = None,
        is_mega: bool = True, 
        is_reusable: bool = True,
        is_verbose = True
    ):
        if torch.cuda.is_available():
          if device == None:
              device = 'cuda'
        else:
          device = 'cpu'
          dtype=torch.float32

        if is_verbose: print("using device", device)
        self.device = device
        self.is_mega = is_mega
        self.is_reusable = is_reusable
        self.dtype = dtype
        self.is_verbose = is_verbose
        self.text_token_count = 64
        self.layer_count = 24 if is_mega else 12
        self.attention_head_count = 32 if is_mega else 16
        self.embed_count = 2048 if is_mega else 1024
        self.glu_embed_count = 4096 if is_mega else 2730
        self.text_vocab_count = 50272 if is_mega else 50264
        self.image_vocab_count = 16415 if is_mega else 16384

        model_name = 'dalle_bart_{}'.format('mega' if is_mega else 'mini')
        dalle_path = os.path.join(models_root, model_name)
        vqgan_path = os.path.join(models_root, 'vqgan')
        if not os.path.exists(dalle_path): os.makedirs(dalle_path)
        if not os.path.exists(vqgan_path): os.makedirs(vqgan_path)
        self.vocab_path = os.path.join(dalle_path, 'vocab.json')
        self.merges_path = os.path.join(dalle_path, 'merges.txt')
        self.encoder_params_path = os.path.join(dalle_path, 'encoder.pt')
        self.decoder_params_path = os.path.join(dalle_path, 'decoder.pt')
        self.detoker_params_path = os.path.join(vqgan_path, 'detoker.pt')

        self.init_tokenizer()
        if is_reusable:
            self.init_encoder()
            self.init_decoder()


    def download_tokenizer(self):
        if self.is_verbose: print("downloading tokenizer params")
        suffix = '' if self.is_mega else '_mini'
        vocab = requests.get(MIN_DALLE_REPO + 'vocab{}.json'.format(suffix))
        merges = requests.get(MIN_DALLE_REPO + 'merges{}.txt'.format(suffix))
        with open(self.vocab_path, 'wb') as f: f.write(vocab.content)
        with open(self.merges_path, 'wb') as f: f.write(merges.content)


    def download_encoder(self):
        if self.is_verbose: print("downloading encoder params")
        suffix = '' if self.is_mega else '_mini'
        urli=MIN_DALLE_REPO + 'encoder{}.pt'.format(suffix)
        !wget -O {self.encoder_params_path} {urli}


    def download_decoder(self):
        if self.is_verbose: print("downloading decoder params")
        suffix = '' if self.is_mega else '_mini'
        urli=MIN_DALLE_REPO + 'decoder{}.pt'.format(suffix)
        !wget -O {self.decoder_params_path} {urli}
    

    def init_tokenizer(self):
        is_downloaded = os.path.exists(self.vocab_path)
        is_downloaded &= os.path.exists(self.merges_path)
        if not is_downloaded: self.download_tokenizer()
        if self.is_verbose: print("intializing TextTokenizer")
        with open(self.vocab_path, 'r', encoding='utf8') as f:
            vocab = json.load(f)
        with open(self.merges_path, 'r', encoding='utf8') as f:
            merges = f.read().split("\n")[1:-1]
        self.tokenizer = TextTokenizer(vocab, merges)


    def init_encoder_h(self):
      with init_empty_weights():
          self.encoder = DalleBartEncoder(
              attention_head_count = self.attention_head_count,
              embed_count = self.embed_count,
              glu_embed_count = self.glu_embed_count,
              text_token_count = self.text_token_count,
              text_vocab_count = self.text_vocab_count,
              layer_count = self.layer_count,
              device=self.device
          )
      init2(self.encoder,self.encoder_params_path)
      self.encoder = self.encoder.cuda()

    def init_encoder(self):
        is_downloaded = os.path.exists(self.encoder_params_path)
        if not is_downloaded: self.download_encoder()
        if self.is_verbose: print("initializing DalleBartEncoder")
        if self.device == 'cuda':
          self.init_encoder_h()
          return
        self.encoder = DalleBartEncoder(
            attention_head_count = self.attention_head_count,
            embed_count = self.embed_count,
            glu_embed_count = self.glu_embed_count,
            text_token_count = self.text_token_count,
            text_vocab_count = self.text_vocab_count,
            layer_count = self.layer_count,
            device=self.device
        ).to(self.dtype).eval()
        params = torch.load(self.encoder_params_path)
        self.encoder.load_state_dict(params, strict=False)
        del params
        self.encoder = self.encoder.to(device=self.device)


    def init_decoder_h(self):
      with init_empty_weights():
          self.decoder = DalleBartDecoder(
              image_vocab_count = self.image_vocab_count,
              attention_head_count = self.attention_head_count,
              embed_count = self.embed_count,
              glu_embed_count = self.glu_embed_count,
              layer_count = self.layer_count,
              device=self.device
          )
      init2(self.decoder,self.decoder_params_path)
      self.decoder = self.decoder.cuda()

    def init_decoder(self):
        is_downloaded = os.path.exists(self.decoder_params_path)
        if not is_downloaded: self.download_decoder()
        if self.is_verbose: print("initializing DalleBartDecoder")
        if self.device == 'cuda':
          self.init_decoder_h()
          return
        self.decoder = DalleBartDecoder(
            image_vocab_count = self.image_vocab_count,
            attention_head_count = self.attention_head_count,
            embed_count = self.embed_count,
            glu_embed_count = self.glu_embed_count,
            layer_count = self.layer_count,
            device=self.device
        ).to(self.dtype).eval()
        params = torch.load(self.decoder_params_path)
        self.decoder.load_state_dict(params, strict=False)
        del params
        self.decoder = self.decoder.to(device=self.device)



In [ ]:
import random

def mkCBemb(seq):
  with net.create_extractor() as ex:
    ex.input("in0", ncnn.Mat(seq).clone())
    hrr, out0 = ex.extract("2")
  del ex
  return out0

def emb2img(emb):
  with net.create_extractor() as ex:
    ex.input("2", emb)
    hrr, out0 = ex.extract("out0")
  del ex
  return Image.fromarray(np.array(out0).astype(np.uint8))

def npmkCBemb(seq):
  with net.create_extractor() as ex:
    ex.input("in0", ncnn.Mat(seq.astype(np.uint32)).clone())
    hrr, out0 = ex.extract("2")
  del ex
  return np.array(out0)


def npemb2img(emb):
  with net.create_extractor() as ex:
    ex.input("2", ncnn.Mat(emb).clone())
    hrr, out0 = ex.extract("out0")
  del ex
  return Image.fromarray(np.array(out0).astype(np.uint8))

def pbla(step,scale):
  ret=[]
  mga=4-(4/scale)
  k=step-1
  for i in range(step):
    ret.append(k+mga*( ((i**2)/k) - i ))
  return ret


def img_float():
  with net.create_extractor() as ex:
      ex.input("in0", ncnn.Mat(curfull[:,sta:endo].reshape(16*(endo-sta)).astype(np.uint32)).clone())
      hrr, out0 = ex.extract("252")
  del ex
  return torch.FloatTensor(np.array(out0)).to(device).unsqueeze(0)


def npimg_float(seq):
  with net.create_extractor() as ex:
      ex.input("in0", ncnn.Mat(seq.astype(np.uint32)).clone())
      hrr, out0 = ex.extract("252")
  del ex
  return np.array(out0)


def hstack(sta,n=16,crop=[]):
  haf=[]
  for k in range(n):
    haf.append(dumped_seqs[sta+k].reshape((16,16))[:,:8])
  if crop:
    sta=crop[0]>>4
    endo=(crop[0]+crop[1])>>4
    return np.hstack(haf)[:,sta:endo].reshape(16*(endo-sta))
  else:
    return np.hstack(haf).reshape(128*n)
    

def fromsteps(liz):
  li=len(liz)-1
  haf=[]
  for n in range(li):
    haf.append(np.fromfile('/content/steps/s%d.bin'%n,dtype=np.uint16).astype(np.int32).reshape((-1,256))[liz[n]].reshape((16,16))[:,:8])
  haf.append(np.fromfile('/content/steps/s%d.bin'%li,dtype=np.uint16).astype(np.int32).reshape((-1,256))[liz[li]].reshape((16,16)))
  return np.hstack(haf).reshape(256 + 128*li)


def mk3x3(idx):
  rowz=[]
  for y in range(3):
    colz=[]
    for x in range(3):
      colz.append(dumped_seqs[idx[y*3+x]].reshape((16,16)))
    rowz.append(np.hstack(colz).reshape(768))
  return np.concatenate(rowz)

def interpo(seq1,seq2,step=30,scale=1.21,outfmt='/content/avif/%02d.png'):
  stp=step-1
  divi=pbla(step,scale)
  em1=npmkCBemb(seq1)
  em2=npmkCBemb(seq2)
  for i in range(step):
    npemb2img((em1*i+em2*(stp-i))/divi[i]).save(outfmt%i)

def interpoC():
  !mkdir /content/avif
  for n in range(candidate_count):
    interpo(dumped_seqs[n],dumped_seqs[n+1])
    !/tmp/ffmpeg/ffmpeg -framerate 18 -i /content/avif/%02d.png -sn -map_metadata -1 -map_chapters -1 -crf 10 -c:v libaom-av1 -aom-params enable-keyframe-filtering=0:enable-tpl-model=1 -lag-in-frames 48 -cpu-used 5 -row-mt 1 -tiles 1x1 -threads 2 -strict experimental -movflags +faststart -flags +cgop -pix_fmt yuv420p10le -c:a libopus -b:a 96k -ac 2 -f webm /content/avif/intp.webm
    os.rename('/content/avif/intp.webm','/content/avif/v%02d.webm'%n)

def showp(n, prt=False):
  daaz=dumped_seqs[n]
  with net.create_extractor() as ex:
    ex.input("in0", ncnn.Mat(daaz.astype(np.uint32)).clone())
    hrr, out0 = ex.extract("out0")
  del ex
  uz=Image.fromarray(np.array(out0).astype(np.uint8))
  uz.save('/content/sample_data/%d.png'%n)
  if prt:
    display(uz)
    return daaz
  else:
    return uz

def showp2(seq):
  with net.create_extractor() as ex:
    ex.input("in0", ncnn.Mat(seq.astype(np.uint32)).clone())
    hrr, out0 = ex.extract("out0")
  del ex
  uz=Image.fromarray(np.array(out0).astype(np.uint8))
  uz.save('/content/sample_data/000.png')
  return uz

def hcopy(tk,left_sta=8):
  len=16-left_sta
  for y in range(16):
    mae=1+y*16
    tk[mae:mae+len]=tk[mae+left_sta:mae+16]

def hcopy_dup(tk,sele,left_sta=8):
  src=sele.expand(candidate_count,-1).T
  len=16-left_sta
  for y in range(16):
    mae=1+y*16
    tk[mae:mae+len]=src[mae+left_sta:mae+16]
  for as0 in range(mindd.layer_count):
    for p in range(4):
      pkan=p*candidate_count
      sle4=attention_state[as0][pkan+candidate_select]
      for j in range(candidate_count):
        attention_state[as0][pkan+j][:]=sle4[:]

def hcopy_dst(src,tk,len=8):
  for y in range(16):
    mae=y*16
    tk[1+mae:1+mae+len]=torch.from_numpy(src[mae:mae+len])

def rumpla():
  global attention_state
  for row_index in range(   ROW_START   ,16):
    print('%x:'%row_index, end='')
    kt=16 * row_index
    for col_index in range(COL_START,16):
      i =  kt + col_index       
      with torch.cuda.amp.autocast(dtype=mindd.dtype):
          image_tokens[i + 1], attention_state = mindd.decoder.forward(
              settings=settings,
              attention_mask=attention_mask,
              encoder_state=encoder_state,
              attention_state=attention_state,
              prev_tokens=image_tokens[i],
              token_index=token_indices[[i]]
          )


def rumplaL(glist):
  global attention_state
  dr=16-len(glist)
  for row_index in range(   dr   ,16):
    print('%x:'%row_index, end='')
    kt=16 * row_index
    for col_index in range( glist[row_index-dr] ,16):
      i =  kt + col_index       
      with torch.cuda.amp.autocast(dtype=mindd.dtype):
          image_tokens[i + 1], attention_state = mindd.decoder.forward(
              settings=settings,
              attention_mask=attention_mask,
              encoder_state=encoder_state,
              attention_state=attention_state,
              prev_tokens=image_tokens[i],
              token_index=token_indices[[i]]
          )

def rudallestuff():
  if not os.path.isfile('/content/model-ru-latest.pt'):
    !git clone https://github.com/sberbank-ai/Real-ESRGAN > /dev/null
    !git clone https://github.com/Jack000/guided-diffusion > /dev/null
    !pip install rudalle > /dev/null
    !pip install -e ./guided-diffusion > /dev/null
    !pip install -r Real-ESRGAN/requirements.txt > /dev/null
    !wget https://huggingface.co/shonenkov/rudalle-utils/resolve/main/RealESRGAN_x2.pth > /dev/null
    !wget https://huggingface.co/shonenkov/rudalle-utils/resolve/main/RealESRGAN_x4.pth > /dev/null
    !wget https://dall-3.com/models/guided-diffusion/ru-dalle/model-ru-latest.pt > /dev/null

def mksettings(top_k0,temperature0,supercondition_factor0):
  return torch.tensor(
    [temperature0, top_k0, supercondition_factor0], 
    dtype=torch.float32,
    device=mindd.device
)
  
def gen0(dr=0,dc=0):
  global ROW_START
  global COL_START
  global dumped_seqs
  ROW_START=dr
  COL_START=dc
  rumpla()
  dumped_seqs=image_tokens[1:].T.to('cpu').numpy().astype(np.uint16)
  with open("/content/ozv.bin",mode='ba+') as f:
    dumped_seqs.tofile(f)


def gen0L(glist):
  global dumped_seqs
  rumplaL(glist)
  dumped_seqs=image_tokens[1:].T.to('cpu').numpy().astype(np.uint16)
  with open("/content/ozv.bin",mode='ba+') as f:
    dumped_seqs.tofile(f)


def chkstz(stz):
  lstz=len(stz)
  if lstz < 2:
    return True
  if stz[1] == '':
    return True
  return False


def gen1():
  torch.set_grad_enabled(False)
  dr=ROW_START
  dc=COL_START
  useL=False
  while os.path.isfile('/content/once.txt'):
    if useL:
      gen0L(glist)
    else:
      gen0(dr,dc)
    try:
      with open('/content/once.txt','rt') as f:
        stz=f.read().replace(' ','').splitlines()
      if stz[0] == '':
        dr=dr
      elif chkstz(stz):
        useL=False
        stz=stz[0].split('@')
        lstz=len(stz)
        if lstz > 0:
          dr=int(stz[0])
        if lstz > 1:
          dc=int(stz[1])
        if lstz > 2:
          settings[1]=float(stz[2])
          if lstz > 3:
            settings[0]=float(stz[3])
          if lstz > 4:
            settings[2]=float(stz[4])
      else:
        glist=[int(x) for x in stz if x.isdigit()][:16]
        useL=True
    except:
      dr=dr
  os.rename('/content/-.txt','/content/once.txt')

def prmp(filltoken=False):
  global seed
  global text_tokens
  global image_tokens
  global attention_mask
  global encoder_state
  global attention_state
  if is_verbose: print("tokenizing text")
  tokens = mindd.tokenizer.tokenize(text, is_verbose=is_verbose)
  if len(tokens) > mindd.text_token_count: 
      tokens = tokens[:mindd.text_token_count]
  if is_verbose: print("{} text tokens".format(len(tokens)), tokens)
  text_tokens = np.ones((2, 64), dtype=np.int32)
  text_tokens[0, :2] = [tokens[0], tokens[-1]]
  text_tokens[1, :len(tokens)] = tokens
  text_tokens = torch.tensor(
      text_tokens, 
      dtype=torch.long, 
      device=mindd.device
  )

  if not mindd.is_reusable: mindd.init_encoder()
  if is_verbose: print("encoding text tokens")
  with torch.cuda.amp.autocast(dtype=mindd.dtype):
      encoder_state = mindd.encoder.forward(text_tokens)
  if not mindd.is_reusable: del mindd.encoder
  torch.cuda.empty_cache()

  if not mindd.is_reusable: mindd.init_decoder()

  with torch.cuda.amp.autocast(dtype=mindd.dtype):
      expanded_indices = [0] * candidate_count + [1] * candidate_count
      text_tokens = text_tokens[expanded_indices]
      encoder_state = encoder_state[expanded_indices].to(token_indices.device)
      attention_mask = text_tokens.not_equal(1).to(token_indices.device)
      if filltoken:
        attention_state = torch.zeros(
            size=(
                mindd.layer_count,
                candidate_count * 4,
                IMAGE_TOKEN_COUNT,
                mindd.embed_count
            ), 
            device=mindd.device
        )
        image_tokens = torch.full(
            (IMAGE_TOKEN_COUNT + 1, candidate_count), 
            mindd.image_vocab_count,
            dtype=torch.long,
            device=mindd.device
        )
      
      if seed == 0:
        seed=random.randint(0, 2**32)
        print('rndseed: '+str(seed)) 
      torch.manual_seed(seed)
        

newstart=True

# PlayGround<br>
User-select outpainting (mini model)
![mini](https://github.com/TabuaTambalam/DalleWebms/blob/main/docs/sbk-mini.png?raw=true)
<br>
mega model:
![mega](https://github.com/TabuaTambalam/DalleWebms/blob/main/docs/sbk-mega.png?raw=true)

In [ ]:
UseMega=True #@param {type:"boolean"}
PrepareRuDalleStuff=False #@param {type:"boolean"}
if newstart:
  mindd = MinDalle(is_mega=UseMega, is_reusable=True)
  token_indices = mindd.decoder.token_indices
  import ncnn
  import gc
  net = ncnn.Net()
  net.opt.use_vulkan_compute = False
  net.load_param(  "/tmp/vq.param"   )  #   "/content/vq3x3.txt"
  net.load_model("/tmp/vq.bin")
  newstart=False
  !nvidia-smi
if PrepareRuDalleStuff:
  t1 = Thread(target = rudallestuff)
  a1 = t1.start()


Make a prompt

In [ ]:
text ="desert oasis merchant market high fantasy book cover painting" #@param {type:"string"}
candidate_count =  4#@param {type:"integer"}
seed= 776677  #@param {type:"integer"}
log_everything = False #@param {type:"boolean"}

'''
555
'''

is_verbose=False


prmp(True)


userselect=[]
userselectN=[0]*128

!rm -rf /content/steps
!mkdir /content/steps
!rm /content/sample_data/*.png

if log_everything:
  with open('/content/steps/prompt.txt','wt') as f:
    f.write(text+'\ntokens='+str(tokens))
  torch.save(attention_mask, '/content/steps/attention_mask.pt')
  torch.save(encoder_state, '/content/steps/encoder_state.pt')


ROW_START =0
COL_START =0
step=0
newprompt=True

change the prompt midway (optional, probably won't produce anything you want)

In [ ]:
text ="Change prompt here (without reset step)" #@param {type:"string"}
seed= 889988  #@param {type:"integer"}
prmp()

Generate loop<br>
(Run this cell multiple times,<br>changing candidate_select & other settings)

In [ ]:
candidate_select  = 1 #@param {type:"integer"}
top_k = 2048 #@param {type:"integer"}
temperature = 3  #@param {type:"integer"}
supercondition_factor = 64 #@param {type:"integer"}

#better not change these
ROW_START =0 
COL_START =7 #@param {type:"integer"}
PreviewLimit=2

settings = torch.tensor(
    [temperature, top_k, supercondition_factor], 
    dtype=torch.float32,
    device=mindd.device
)


if newprompt:
  ROW_START=0
  COL_START=0
elif candidate_select < candidate_count:
  userselectN[step]+=candidate_select
  userselect.append(dumped_seqs[candidate_select].reshape((16,16))[:,:8])
  hcopy_dup(image_tokens,image_tokens.T[candidate_select])
  step+=1
else:
  userselectN[step]+=candidate_count
  if step == 0:
    newprompt=True
    ROW_START=0
    COL_START=0


rumpla()

fnapfix='/content/steps/s%d'%step
dumped_seqs=image_tokens[1:].T.to('cpu').numpy().astype(np.uint16)
with open(fnapfix+'.bin',mode='ba+') as f:
  dumped_seqs.tofile(f)


if log_everything:
  torch.save(settings, fnapfix+'_settings.pt')
  #torch.save(attention_state, fnapfix+'_attention_state.pt')

print('\n')
if newprompt:
  newprompt=False
  for n in range(candidate_count):
    print('=====Init%d====='%n,end='')
    display(showp(n))
else:
  syz=len(userselect)
  if syz > PreviewLimit:
    tview=userselect[syz-PreviewLimit:]
    syz=PreviewLimit
  else:
    tview=userselect
  bdfna='-'.join(str(x) for x in userselectN[:step])
  for n in range(candidate_count):
    print('=====Next%d====='%n,end='')
    display(showp2( np.hstack(tview+[dumped_seqs[n].reshape((16,16))]).reshape(256+128*syz) ))
    dfna='/content/sample_data/'+bdfna+'-'+str(n)+'.png'
    if os.path.isfile(dfna):
      os.remove(dfna)
    os.rename('/content/sample_data/000.png',dfna)


newprompt=False

Finish gen

In [ ]:
candidate_select=1 #@param {type:"integer"}
userselect.append(dumped_seqs[candidate_select].reshape((16,16)))
userselectN[step]+=candidate_select
step+=1


userselectN=userselectN[:step]
curfull=np.hstack(userselect).astype(np.uint32)
print(userselectN)
showp2(curfull.reshape(128+128*len(userselect)))

================<br>Crop & Re-decode

In [ ]:
Left=219 #@param {type:"integer"}
Width=355 #@param {type:"integer"}

sta=Left>>4
endo=(Left+Width)>>4
showp2(curfull[:,sta:endo].reshape(16*(endo-sta)))

# Tools

Reload ncnndec<br>
(when memory leaks, and will make ncnn faster on GPU somethimes)<br>
DO NOT use height_free for this interactive demo

In [ ]:
UseVulkan=False #@param {type:"boolean"}
Height_free=False #@param {type:"boolean"}

try:
  del net
except:
  import ncnn
  import gc
  print('load only')

gc.collect()

net = ncnn.Net()
net.opt.use_vulkan_compute = UseVulkan
net.opt.use_fp16_packed = False
net.opt.use_fp16_storage = False
net.opt.use_fp16_arithmetic = False
if Height_free:
  net.load_param("/tmp/vq_vert.param")
else:
  net.load_param(  "/tmp/vq.param"   )  #   "/content/vq3x3.txt"
net.load_model("/tmp/vq.bin")

pack steps

In [ ]:
!7z a /content/pk.7z /content/steps
if len(userselectN) < 128:
  dfna=','.join(str(x) for x in userselectN)
  os.rename('/content/pk.7z','/content/'+dfna+'  .7z')

In [ ]:
!7z x pk.7z
curfull=fromsteps([ 6,5,1,1,2,2,2,2,2,2,1,2    ])
display(showp2(curfull))
curfull=curfull.reshape((16,-1))

In [ ]:
%%html
<html><body><style>
FL { 
position: relative;
display: inline-block;
background-color: #808;
width: 1024px;
min-height: 1024px;
}

NG { 
min-width: 64px;
min-height: 64px;
position: relative;
display: inline-block;
text-align: center;
line-height : 60px;
font-weight: bold;
font-size: 140%;
color: #f00;
}

.ckb {
	position: absolute;
	top: 0px;
	left: 0px;
	min-width: 1024px;
	-webkit-mask: url('data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEAAQMAAABmvDolAAAABlBMVEUAAAD///+l2Z/dAAAAAXRSTlMAQObYZgAAAGhJREFUaN7t2rENgDAMRUFvyTgMxDwswRCEDT5IpEiis9voCuuVqfYyXx7scavdaQldhTx1HXnr3PLqYS5BDwQ9EPRA0ANBDwQ9EPQw2iXXEPRA0ANBDwQ9EPRA0ANBD6Ndcg3h5++FB6kp5Ka9HRXlAAAAAElFTkSuQmCC') 0% 0% / 100%;
}
</style>
<FL><img class="ckb" src="https://localhost:8233/-1.png"></FL><br>
</body>
<script>
var nfo=new Array(256); 
// Token space visualizer =False  #@param {type:"boolean"}

function asstok()
{
	var atr=this.value.split(' ').filter(element => element);
	for(var i=0;i<256;i++)
	{
		nfo[i].innerHTML=atr[i];
	}


}

function setup()
{
	var bz=document.getElementsByTagName('FL')[0];

	for(var y=0;y<16;y++)
	{
		var yp=y*16;
		for(var x=0;x<16;x++)
		{
			var idx=yp+x;
			var ngp=document.createElement('NG');
			ngp.innerHTML=idx;
			nfo[idx]=ngp;
			bz.appendChild(ngp);
			
		}
		
	}

	var txta=document.createElement('textarea');
	txta.cols=150;
	txta.rows=30;
	document.body.appendChild(txta);
	txta.ondblclick=asstok;
	

	

}
setup()

</script></html>

In [ ]:
#from google.colab import output
!nohup python3 -m http.server -d /content/sample_data/ 8233 > izh.txt &
#output.serve_kernel_port_as_window(8233, path='')

# Omake: Lock toprows/leftcols generate with candidates
Lock top rows: 
![interpo mk2plus](https://github.com/TabuaTambalam/DalleWebms/blob/main/docs/room.gif?raw=true)
mini model: 
![interpo superman, mini model](https://github.com/TabuaTambalam/DalleWebms/blob/main/docs/interpo_mini.gif?raw=true)


Lock top rows

In [ ]:
candidate_select=11 #@param {type:"integer"}
top_k = 2048 #@param {type:"integer"}
temperature = 3  #@param {type:"integer"}
supercondition_factor = 64 #@param {type:"integer"}

settings = torch.tensor(
    [temperature, top_k, supercondition_factor], 
    dtype=torch.float32,
    device=mindd.device
)

stepping = 2 #@param {type:"integer"}
r_ROW_START =-1 #@param {type:"integer"}
r_COL_START =-1 #@param {type:"integer"}

if ROW_START > 15:
  print('No more')
  assert False

if newprompt:
  ROW_START=0
  COL_START=0
else:
  if r_ROW_START >=0:
    ROW_START=r_ROW_START>>4
  if r_COL_START >=0:
    COL_START=r_COL_START>>4
  if candidate_select < candidate_count:
    image_tokens[:]=image_tokens.T[candidate_select].expand(candidate_count,-1).T[:]
    for as0 in range(mindd.layer_count):
      for p in range(4):
        pkan=p*candidate_count
        sle4=attention_state[as0][pkan+candidate_select]
        for j in range(candidate_count):
          attention_state[as0][pkan+j][:]=sle4[:]
    fna='/content/sample_data/'+str(candidate_select)
    dfna=fna+'h'+str(ROW_START<<4)+'.png'
    if os.path.isfile(dfna):
      os.remove(dfna)
    os.rename(fna+'.png',dfna)
  else:
    ROW_START-=stepping


rumpla()
ROW_START+=stepping
  


dumped_seqs=image_tokens[1:].T.to('cpu').numpy().astype(np.uint16)
with open("/content/ozv.bin",mode='ba+') as f:
  dumped_seqs.tofile(f)


msg='=== U rike dis top'+str(ROW_START<<4)+'x256? %d===='

print('\n')
newprompt=False
for n in range(candidate_count):
  print(msg%n,end='')
  display(showp(n))



Load external seq (probably won't produce anything you want)

In [ ]:
#!wget https://github.com/TabuaTambalam/DalleWebms/releases/download/0.1/ozv.bin

In [ ]:
seq_bin ="/content/ozv.bin" #@param {type:"string"}
seq_idx =  21#@param {type:"integer"}
seq=np.fromfile(seq_bin,dtype=np.uint16).astype(np.int64).reshape((-1,256))[seq_idx]
image_tokens[1:]=torch.from_numpy(seq).expand(candidate_count,-1).T
newprompt=False
showp2(seq)

P.S. Actually has no knowledge about pytorch,<br>
hope someone can make this shorter/less operates/less copying:<br>
```
image_tokens[ : ]=image_tokens.T[candidate_select].expand(candidate_count,-1).T[ : ]
```

And this less loop:
```
  for as0 in range(mindd.layer_count):
    for p in range(4):
      pkan=p*candidate_count
      sle4=attention_state[as0][pkan+candidate_select]
      for j in range(candidate_count):
        attention_state[as0][pkan+j][:]=sle4[:]
```


Lock left cols

In [ ]:
candidate_select=0 #@param {type:"integer"}
top_k = 2048 #@param {type:"integer"}
temperature = 3  #@param {type:"integer"}
supercondition_factor = 64 #@param {type:"integer"}

settings = torch.tensor(
    [temperature, top_k, supercondition_factor], 
    dtype=torch.float32,
    device=mindd.device
)

stepping = 2 #@param {type:"integer"}
r_ROW_START =-1 #@param {type:"integer"}
r_COL_START =-1 #@param {type:"integer"}

if COL_START > 15:
  print('No more')
  assert False



if newprompt:
  ROW_START=0
  COL_START=0
else:
  if r_ROW_START >=0:
    ROW_START=r_ROW_START>>4
  if r_COL_START >=0:
    COL_START=r_COL_START>>4
  if candidate_select < candidate_count:
    image_tokens[:]=image_tokens.T[candidate_select].expand(candidate_count,-1).T[:]
    for as0 in range(mindd.layer_count):
      for p in range(4):
        pkan=p*candidate_count
        sle4=attention_state[as0][pkan+candidate_select]
        for j in range(candidate_count):
          attention_state[as0][pkan+j][:]=sle4[:]
    fna='/content/sample_data/'+str(candidate_select)
    dfna=fna+'w'+str(COL_START<<4)+'.png'
    if os.path.isfile(dfna):
      os.remove(dfna)
    os.rename(fna+'.png',dfna)
  else:
    COL_START-=stepping



rumpla()
COL_START+=stepping


dumped_seqs=image_tokens[1:].T.to('cpu').numpy().astype(np.uint16)
with open("/content/ozv.bin",mode='ba+') as f:
  dumped_seqs.tofile(f)


msg='=== U rike dis left256x'+str(COL_START<<4)+'? %d===='

print('\n')
newprompt=False
for n in range(candidate_count):
  print(msg%n,end='')
  display(showp(n))



# Infinite similar gen

In [ ]:
candidate_select=0 #@param {type:"integer"}
top_k = 2048 #@param {type:"integer"}
temperature = 3  #@param {type:"integer"}
supercondition_factor = 64 #@param {type:"integer"}


r_ROW_START =35 #@param {type:"integer"}
r_COL_START =0 #@param {type:"integer"}

settings=mksettings(top_k,temperature,supercondition_factor)





if newprompt:
  newprompt=False
  gen0()
  for n in range(candidate_count):
    print('init%d'%n,end='')
    display(showp(n))
elif candidate_select < candidate_count:
  image_tokens[:]=image_tokens.T[candidate_select].expand(candidate_count,-1).T[:]
  for as0 in range(mindd.layer_count):
    for p in range(4):
      pkan=p*candidate_count
      sle4=attention_state[as0][pkan+candidate_select]
      for j in range(candidate_count):
        attention_state[as0][pkan+j][:]=sle4[:]
  ROW_START=r_ROW_START>>4
  COL_START=r_COL_START>>4
  print("infinite gen started in thread")
  t1 = Thread(target = gen1)
  a1 = t1.start()
else:
  gen0()
  for n in range(candidate_count):
    print('init%d'%n,end='')
    display(showp(n))



(infinite gen) Show last (0~candidate_count) gen

In [ ]:
showp(-1, True)

(infinite gen) change top_k .etc midway<br>
change the prompt midway with that `889988` one<br>
rename `once.txt` to `-.txt` to stop the infinite gen<br>
change `(skip top n rows)@(skip left n cols)` in token space in `once.txt` e.g. `4@0` or `0@2`

In [ ]:
top_k = 2048 #@param {type:"integer"}
temperature = 3  #@param {type:"integer"}
supercondition_factor = 64 #@param {type:"integer"}
settings[0]=temperature
settings[1]=top_k
settings[2]=supercondition_factor

# RuDalle's ESRGAN

In [ ]:
rudallestuff()
sys.path.append('./Real-ESRGAN')
sys.path.append('./guided-diffusion')

UseGuidedDiffusion=True #@param {type:"boolean"}
save_path='/content/sample_data/'  #@param {type:"string"}

import os
from realesrgan import RealESRGAN
from torch.nn import functional as F
from rrdbnet_arch import RRDBNet
from utils_sr import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

if UseGuidedDiffusion:
  torch.set_grad_enabled(True)
  model_params = {
      'attention_resolutions': '32, 16, 8',
      'class_cond': False,
      'diffusion_steps': 2000,
      'rescale_timesteps': True,
      'timestep_respacing': '', #'ddim2000',
      'image_size': 256,
      'learn_sigma': True,
      'noise_schedule': 'linear',
      'num_channels': 256,
      'num_head_channels': 64,
      'num_res_blocks': 2,
      'resblock_updown': True,
      'use_fp16': True,
      'use_scale_shift_norm': True,
      'emb_condition': True
  }
  import PIL
  from einops import rearrange
  from rudalle import get_vae
  from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults
  import torch.nn as nn
  from torch.nn import functional as F
  from torchvision import transforms as T
  from torchvision.transforms import functional as TF
  import torchvision

  resr = RealESRGAN(device, scale=2)
  resr.load_weights('/content/RealESRGAN_x2.pth')


  if not torch.cuda.is_available():
    model_params['use_fp16']=False

  model_config = model_and_diffusion_defaults()
  model_config.update(model_params)

  diffusion_decoder_model, diffusion = create_model_and_diffusion(**model_config)
  diffusion_decoder_model.load_state_dict(torch.load('/content/model-ru-latest.pt', map_location='cpu'))
  diffusion_decoder_model.requires_grad_(False).eval().to(device)

  for name, param in diffusion_decoder_model.named_parameters():
      if 'qkv' in name or 'norm' in name or 'proj' in name:
          param.requires_grad_()

  if model_config['use_fp16']:
      diffusion_decoder_model.convert_to_fp16()

  vae = get_vae(dwt=False).to(device)
  embedding = nn.Embedding.from_pretrained(vae.model.quantize.embed.weight)
else:
  resr = RealESRGAN(device, scale=4)
  resr.load_weights('/content/RealESRGAN_x4.pth')


def preprocess(img, target_image_size=256):
    s = min(img.size)
    if s < target_image_size:
        raise ValueError(f'min dim for image {s} < {target_image_size}')
    r = target_image_size / s
    s = (round(r * img.size[1]), round(r * img.size[0]))
    img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
    img = TF.center_crop(img, output_size=2 * [target_image_size])
    img = torch.unsqueeze(T.ToTensor()(img), 0)
    return img

def postesrgan(res):
  sr_image = res.permute((0,2,3,1)).clamp_(0, 1).cpu()
  sr_img=Image.fromarray((sr_image[0].numpy()*255).astype(np.uint8))
  sr_img.save(save_path+'000_x4.png')
  return sr_img

def Thrd_show(pp,fna):
  ymg = Image.fromarray(((pp.transpose((1,2,0))+1)*127.5).clip(0,255).astype(np.uint8))
  ymg.save(save_path+fna+'.png')
  display(ymg)
  np.savez(save_path+fna+'.npz',pp)

def diffusion_decode(z,seed=None,stop_at = 1000,logstep=50):
    res = []
    image_index = 1          
    diffusion_batch_size = 1
    diffusion_num_batches = 1
    
    cblen=z.shape[1]
    if seed is not None:
        torch.manual_seed(seed)

  
    

    target_embeds, weights = [], []
    img_seq = z.to(device)

    b, n = 1, cblen # z.shape
    z = embedding(img_seq)

    embeds = rearrange(z, 'b (h w) c -> b c h w', h = 32)
    side_y=embeds.shape[2]*16
    side_x=embeds.shape[3]*16
    embeds = embeds.repeat(diffusion_batch_size, 1, 1, 1)
    

    cur_t = None

    if model_config['timestep_respacing'].startswith('ddim'):
        sample_fn = diffusion.ddim_sample_loop_progressive
    else:
        sample_fn = diffusion.p_sample_loop_progressive

    for i in range(diffusion_num_batches):
        cur_t = diffusion.num_timesteps - 1

        samples = sample_fn(
            diffusion_decoder_model,
            (diffusion_batch_size, 3, side_y, side_x),
            clip_denoised=False,
            model_kwargs={'image_embeds': embeds},
            cond_fn=None,
            progress=True,
        )
        for j, sample in enumerate(samples):
          cur_t -= 1
          isending=(cur_t == -1 or j > stop_at)
          if j % logstep == 0 or isending:
              for k, image in enumerate(sample['pred_xstart']):
                t1=Thread(target=Thrd_show, args=(image.cpu().numpy(),'%d_%d'%(j,k),)) 
                a1 = t1.start()
                if isending:
                  res.append(image.add(1).div(2))
              if isending:
                return res
    return res

works with `curfull` loaded

In [ ]:
Left=0 #@param {type:"integer"}
Width=640 #@param {type:"integer"}

sta=Left>>4
endo=(Left+Width)>>4

with torch.no_grad():
  res = resr.model(img_float())

postesrgan(res)

works with any image

In [ ]:
FileName='/content/sample_data/7-21-11.png' #@param {type:"string"}
lr_image=np.array(Image.open(FileName).convert('RGB'))
batch_size=4
patches_size=200
padding=28
pad_size=28

scale = resr.scale
device = resr.device
lr_image = np.array(lr_image)
lr_image = pad_reflect(lr_image, pad_size)

patches, p_shape = split_image_into_overlapping_patches(lr_image, patch_size=patches_size, 
                                                        padding_size=padding)
img = torch.FloatTensor(patches/255).permute((0,3,1,2)).to(device).detach()

with torch.no_grad():
    res = resr.model(img[0:batch_size])
    for i in range(batch_size, img.shape[0], batch_size):
        res = torch.cat((res, resr.model(img[i:i+batch_size])), 0)

sr_image = res.permute((0,2,3,1)).clamp_(0, 1).cpu()
np_sr_image = sr_image.numpy()

padded_size_scaled = tuple(np.multiply(p_shape[0:2], scale)) + (3,)
scaled_image_shape = tuple(np.multiply(lr_image.shape[0:2], scale)) + (3,)
np_sr_image = stich_together(np_sr_image, padded_image_shape=padded_size_scaled,
                        target_shape=scaled_image_shape, padding_size=padding * scale)
sr_img = (np_sr_image*255).astype(np.uint8)
sr_img = unpad_image(sr_img, pad_size*scale)
sr_img=Image.fromarray(sr_img)
sr_img.save(FileName[:-4]+"_x4.png")
sr_img

Rudalle GD

In [ ]:
Left=0 #@param {type:"integer"}
Width=512 #@param {type:"integer"}
StopAt=2000 #@param {type:"integer"}
LogAt=50 #@param {type:"integer"}

UseSeq=-999999  #@param {type:"integer"}

if UseSeq > -999990:
  codebooks = vae.get_codebook_indices(torch.FloatTensor(npimg_float(dumped_seqs[UseSeq])).to(device).unsqueeze(0))
else:
  sta=Left>>4
  endo=(Left+Width)>>4
  codebooks = vae.get_codebook_indices(img_float())
resGD = diffusion_decode(codebooks,seed=None,stop_at=StopAt,logstep=LogAt)


In [ ]:
postesrgan(resr.model(resGD[-1].unsqueeze(0)))

release ram if necessary

In [ ]:
del mindd
del net
gc.collect()

Tool: load dumped_seqs

In [ ]:
seqbin='ozv_lock.bin' #@param {type:"string"}
dumped_seqs=np.fromfile(seqbin,dtype=np.uint16).astype(np.int32).reshape((-1,256))

Tool: test cb

In [ ]:
import torchvision
torchvision.transforms.functional.to_pil_image(vae.decode(codebooks)[0]).convert('RGB')